# 00 라이브러리

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

# Selenium
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException,InvalidSessionIdException

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

import pandas as pd
import time

# Send mail
from send_mail import send_message

from IPython.display import clear_output
import re

# 01 기본설정

### 메일설정

In [ ]:
to_mail = 'applewoods@g.skku.edu'
msg_title = '네이버 뉴스 크롤링 결과'

### url 및 경로설정

In [ ]:
url = 'https://www.naver.com/'
driver_path = './data/chromedriver.exe'
save_path = './data/raw/'

### 검색어 설정

In [ ]:
search_keywords = " "

### 날짜설정

In [ ]:
START_DATE = "2017.10.06"
END_DATE = "2017.10.31"
PERIOD = pd.date_range(START_DATE, END_DATE)
DATE = [str(date.date()).replace("-",".") for date in PERIOD]
date_len = len(DATE)

### 크롤링에 필요한 변수

In [ ]:
colums_list = ["Date", "Company", "Title", "Body"]
NEWS_DATA = []
count = 583
pass_amount = 0
news_amount = 0

# 02 Crawling

In [ ]:
try:
    for i in range(0, date_len):
        # 날짜
        start_date = DATE[i]
        end_date = DATE[i]
        
        print("Crawling Start\n날짜:",start_date,"~",end_date)
        clear_output(wait=True)
        
        # googledriver 세팅
        driver = webdriver.Chrome(driver_path)
        driver.implicitly_wait(20)
        driver.maximize_window()
        driver.get(url)
        driver.implicitly_wait(20)

        # 검색어 입력
        driver.find_element_by_name("query").send_keys(search_keywords)
        driver.find_element_by_id("search_btn").click()
        driver.implicitly_wait(3)

        # 뉴스탭 누르기
        driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[1]/div[2]/div[1]/div/ul/li[4]/a").click()
        time.sleep(3)

        # 기간설정
        driver.find_element_by_id("search_option_button").click()
        driver.implicitly_wait(3)
        driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[2]/div/ul/li[2]/a").click()
        driver.implicitly_wait(3)
        driver.find_element_by_id("news_input_period_begin").send_keys(start_date)
        driver.find_element_by_id("news_input_period_end").send_keys(end_date)
        driver.implicitly_wait(3)
        driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[2]/div/ul/li[2]/div/div[2]/span/button").click()
        driver.implicitly_wait(3)
        driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/div[1]/div[1]/div[1]/a[3]").click()
        driver.implicitly_wait(3)
        
        try:
            # 뉴스데이터 크롤링
            while True:
                NEWS_DATA.clear()
                NEWS = driver.find_elements_by_css_selector("#main_pack > section.sc_new.sp_nnews._prs_nws > div > div.group_news > ul > li")

                # DATA CRAWLING
                for n in NEWS:
                    print("Crawling...\n날짜:",start_date,"~",end_date,"\n크롤링 갯수:",news_amount,", pass 된 갯수:",pass_amount, ", 크롤링 중인 페이지:", count)
                    clear_output(wait=True)

                    info = n.find_elements_by_class_name("info")
                    driver.implicitly_wait(5)
                    if len(info) == 3:
                        company = info[0].text
                        date = info[1].text
                        info[-1].click()
                        driver.switch_to.window(driver.window_handles[-1])
                        driver.implicitly_wait(5)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        time.sleep(0.3)

                        try:
                            title = soup.find(attrs={"id":"articleTitle"}).text
                            news = soup.find(attrs={"id":"articleBodyContents"}).text
                            NEWS_DATA.append([date, company, title, news])
                            news_amount += 1
                        except:
                            pass_amount += 1
                            pass   

                        driver.close()
                        driver.implicitly_wait(10)
                        driver.switch_to.window(driver.window_handles[0])
                        driver.implicitly_wait(10)               

                # DATA SAVE
                df = pd.DataFrame(NEWS_DATA, columns=colums_list)
                df.to_csv(save_path+"NaverNews{0}.csv".format(count), index=False)
                time.sleep(0.5)
                count += 1

                print("Save Success\n날짜:",start_date,"~",end_date,"\n크롤링 갯수:",news_amount,", pass 된 갯수:",pass_amount, ", 크롤링 중인 페이지:", count)
                clear_output(wait=True)

                # NEXT PAGE
                page = len(driver.find_elements_by_css_selector("#main_pack > section > div > div.group_news > ul > li"))
                if page == 10:
                    driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/div[1]/div[2]/div/a[2]").click()
                    time.sleep(1)
                else:
                    break
                    
        except ElementClickInterceptedException as e:
            pass
        # googledriver 종료
        driver.quit()
        time.sleep(3)
        
except FileNotFoundError as e:
    print(e)
    msg = "Error!!\n 파일경로 확인!!"
    ms = message(to_mail, from_mail, msg_title, msg)
    ms()
    
except KeyboardInterrupt as k:
    print(k)
    
except NoSuchElementException as n:
    print("Error: NoSuchElementException")
    print(n)
    msg = "Error!!\n{0} ~ {1} Naver News 크롤링 완료\n크롤링 갯수: {2}, pass 된 갯수: {3}".format(start_date, end_date, news_amount, pass_amount)
    ms = message(to_mail, from_mail, msg_title, msg)
    ms()
    
except ElementClickInterceptedException as e:
    print("Error: ElementClickInterceptedException")
    print(e)
    msg = "Error!!\n{0} ~ {1} Naver News 크롤링 완료\n크롤링 갯수: {2}, pass 된 갯수: {3}".format(start_date, end_date, news_amount, pass_amount)
    ms = message(to_mail, from_mail, msg_title, msg)
    ms()
    
except InvalidSessionIdException as i:
    print("Error: InvalidSessionIdException")
    print(i)
    msg = "Error!!\n{0} ~ {1} Naver News 크롤링 완료\n크롤링 갯수: {2}, pass 된 갯수: {3}".format(start_date, end_date, news_amount, pass_amount)
    ms = message(to_mail, from_mail, msg_title, msg)
    ms()

Save Success
날짜: 2017.10.31 ~ 2017.10.31 
크롤링 갯수: 7478 , pass 된 갯수: 5401 , 크롤링 중인 페이지: 3592


In [ ]:
print("크롤링이 완료되었습니다.")
print("크롤링 결과:",news_amount,", pass 된 갯수:",pass_amount)
msg = "{0} ~ {1} Naver News 크롤링 완료\n크롤링 갯수: {2}, pass 된 갯수: {3}".format(start_date, end_date, news_amount, pass_amount)
ms = message(to_mail, from_mail, msg_title, msg)
ms()

크롤링이 완료되었습니다.
크롤링 결과: 7478 , pass 된 갯수: 5401
